In [18]:
import sys
sys.path.append('mnt/sw/nix/store/i4qqrrhx6cjsr6r2vl06h9fwbzc9qs8p-texlive-20210325/bin/x86_64-linux/')

import numpy as np
from numpy import linalg as la
from scipy.linalg import norm

import matplotlib
import matplotlib.pyplot as plt
plt.style.use('~/matplotlib.mplstyle')

from astropy import units as u
#import pynbody
#import healpy as hp
#from healpy.newvisufunc import projview, newprojplot

sys.path.append("../scripts/src/")
sys.path.append("/mnt/home/ecunningham/python/")
import halo_analysis as halo

#import nba

#import io_gizmo_pynbody as ga
#import halo_analysis as halo

# 
#import pynbody_routines as pr 
#import io_gizmo_pynbody as fa
#import plotting as pl



In [4]:
def return_tracked_pos(halo_tree, tr_ind_at_init, pynbody_halo=False, nsnaps=300):
    # Adapted from Arpit's function
    h_index = tr_ind_at_init
    tree_ind = []

    for _ in range(nsnaps):
        tree_ind.append(h_index)
        h_index = halo_tree['descendant.index'][h_index]
    tree_ind = np.array(tree_ind)
    position = halo_tree['host.distance'][tree_ind]
    nsnaps = halo_tree['snapshot'][tree_ind]
    mass = halo_tree['mass'][tree_ind]
    velocity = halo_tree['host.velocity'][tree_ind]
    #vel_rad = halt['host.velocity.rad'][tree_ind]
    #vel_tan = halt['host.velocity.tan'][tree_ind]
    if pynbody_halo == True:
        sat = {'position': position,
             'mass': mass,
             'velocity': velocity,
          }
        return pr.pynbody_satellite(sat)

    elif pynbody_halo == False:

        return {'position': position,
             'snaps' : nsnaps,
             'mass' : mass,
             'velocity' : velocity,
                }


In [113]:
sim = 'm12c'
sim_directory = "/mnt/ceph/users/firesims/fire2/metaldiff/{}_res7100/".format(sim)

#sat_path = '/mnt/home/ecunningham/ceph/latte/{}_res7100/massive_stream/dm_inds.npy'.format(sim)
#subs_path = '/mnt/home/nico/ceph/FIRE/{}_385_unbound_dark_indices.npy'.format(sim)
#subs_ids = np.load(subs_path)
#stars_path = '/mnt/home/ecunningham/ceph/latte/{}_res7100/massive_stream/new_z0_inds.npy'.format(sim)
#stars_ids = np.load(stars_path)
smax = 300 # snapshot at peak mass
sat_index_peak_mass = -2
nsnaps = 600-smax+1
sat_ids = np.load(sat_path)

In [114]:
halt = halo.io.IO.read_tree(simulation_directory=sim_directory)


# in utilities.simulation.Snapshot():
* reading:  mnt/ceph/users/firesims/fire2/metaldiff/m12b_res7100/snapshot_times.txt

* read 19882039 halos from:  mnt/ceph/users/firesims/fire2/metaldiff/m12b_res7100/halo/rockstar_dm/catalog_hdf5/tree.hdf5



In [141]:
def get_catids_sat_subhalos(smax, sipm):
    """
    sipm : satellite index peak mass
    """
    
    halt = halo.io.IO.read_tree(simulation_directory=sim_directory)
    snap_peak_mass = np.where(halt['snapshot'] == smax)[0]
    # Find satellite at snap of max mass given its sat_index_peak_mass
    sat_id_peak_mass = np.argsort(halt['mass'][snap_peak_mass])[sipm]
    print(sipm, halt['mass'][sat_id_peak_mass], np.sort(halt['mass'][snap_peak_mass])[-2]/1E12)
    #print(sat_id_peak_mass, snap_peak_mass[sipm], halt['central.local.index'][snap_peak_mass])
    # Find subhalos that have the central local index of the satellite
    print(halt['mass'][sat_id_peak_mass])
    subhalos_sat = np.where((halt['central.local.index'][snap_peak_mass] == snap_peak_mass[sipm]))[0]# & (halt['mass'][snap_peak_mass]>1e6))[0]
    nsub_sat = len(subhalos_sat)
    print(subhalos_sat)
    print('-> Number of satellite subhalos at peak mass identified by halo finder {} at snapshot {}'.format(nsub_sat, smax))
    print('-> Total number of subhalos at snap {}: {}'.format(smax, len(snap_peak_mass)))
    # Find tree subhalos of satellite index at peak mass
    sat_indices = np.zeros(nsub_sat, dtype=int)
    sat_indices = snap_peak_mass[subhalos_sat]
    cat_idx = np.zeros((nsnaps, nsub_sat), dtype=int)
    cat_idx[0] = halt['catalog.index'][sat_indices]
    for k in range(0, nsnaps-1):
        sat_indices = halt['descendant.index'][sat_indices]
        cat_idx[k+1] = halt['catalog.index'][sat_indices]
    return cat_idx

In [142]:
#hsub = pr.pynbody_subhalos(m12_subhalos) # this just give me satellite information 
subhalos_ids = get_catids_sat_subhalos(smax, sat_index_peak_mass)
print("-> Removing satellite subhalos")



# in utilities.simulation.Snapshot():
* reading:  mnt/ceph/users/firesims/fire2/metaldiff/m12b_res7100/snapshot_times.txt

* read 19882039 halos from:  mnt/ceph/users/firesims/fire2/metaldiff/m12b_res7100/halo/rockstar_dm/catalog_hdf5/tree.hdf5

-2 773988.6 0.205356122112
773988.6
[]
-> Number of satellite subhalos at peak mass identified by halo finder 0 at snapshot 300
-> Total number of subhalos at snap 300: 42970
-> Removing satellite subhalos


In [97]:
sim_directory

'/mnt/ceph/users/firesims/fire2/metaldiff/m12f_res7100/'

In [149]:
#sims = ['m12b', 'm12c', 'm12f', 'm12i', 'm12m']
sims = ['m12w']
snap_in = [215, 286, 224]
i=0
for snaps in snap_in:
    sim_directory = "/mnt/ceph/users/firesims/fire2/metaldiff/{}_res7100/".format('m12w')
    m12_subhalos = halo.io.IO.read_catalogs('index', snaps, sim_directory);
    print(sims, m12_subhalos['mass.200m'][m12_subhalos.prop('host.distance.total')==0])
    i+=1


# in utilities.simulation.Snapshot():
* reading:  mnt/ceph/users/firesims/fire2/metaldiff/m12w_res7100/snapshot_times.txt

  using snapshot index = 215, redshift = 1.490

* read 56909 halos from:  mnt/ceph/users/firesims/fire2/metaldiff/m12w_res7100/halo/rockstar_dm/catalog_hdf5/halo_215.hdf5

# in halo_analysis.halo_io.Particle():
* read 56909 halos, 136 have star particles, from:  mnt/ceph/users/firesims/fire2/metaldiff/m12w_res7100/halo/rockstar_dm/catalog_hdf5/star_215.hdf5
* assigning primary host and coordinates wrt it to halo catalog...  finished

['m12w'] [3.0222057e+11]

# in utilities.simulation.Snapshot():
* reading:  mnt/ceph/users/firesims/fire2/metaldiff/m12w_res7100/snapshot_times.txt

  using snapshot index = 286, redshift = 0.944

* read 54977 halos from:  mnt/ceph/users/firesims/fire2/metaldiff/m12w_res7100/halo/rockstar_dm/catalog_hdf5/halo_286.hdf5

# in halo_analysis.halo_io.Particle():
* read 54977 halos, 107 have star particles, from:  mnt/ceph/users/firesims/fi

In [99]:
[m12_subhalos.prop('host.distance')==0]

[array([[False, False, False],
        [False, False, False],
        [False, False, False],
        ...,
        [False, False, False],
        [False, False, False],
        [False, False, False]])]

In [104]:
#sims = ['m12b', 'm12c', 'm12f', 'm12i', 'm12m', 'm12r', 'm12w']
#for sim in sims:
#    sim_directory = "/mnt/ceph/users/firesims/fire2/metaldiff/{}_res7100/".format(sim)
#    m12_subhalos = halo.io.IO.read_catalogs('index', 600, sim_directory);
#    print(sim, m12_subhalos['mass.200m'][m12_subhalos.prop('host.distance.total')==0])

In [31]:
sims = ['m12c']
for sim in sims:
    sim_directory = "/mnt/ceph/users/firesims/fire2/metaldiff/{}_res7100/".format(sim)
    m12_subhalos = halo.io.IO.read_catalogs('index', 492, sim_directory);
    print(sim, m12_subhalos['mass.200m'][m12_subhalos.prop('host.distance.total')==0])


# in utilities.simulation.Snapshot():
* reading:  mnt/ceph/users/firesims/fire2/metaldiff/m12c_res7100/snapshot_times.txt

  using snapshot index = 492, redshift = 0.187

* read 108913 halos from:  mnt/ceph/users/firesims/fire2/metaldiff/m12c_res7100/halo/rockstar_dm/catalog_hdf5/halo_492.hdf5

# in halo_analysis.halo_io.Particle():
* read 108913 halos, 128 have star particles, from:  mnt/ceph/users/firesims/fire2/metaldiff/m12c_res7100/halo/rockstar_dm/catalog_hdf5/star_492.hdf5
* assigning primary host and coordinates wrt it to halo catalog...  finished

m12c [7.8189455e+11]


In [42]:
0.8/3.5

0.2285714285714286

### 

halo | halo catalog | Wetzel+22
:=============================:
m12b 1.1821225e+12 | 1.43e+12 
m12c 1.126225e+12 | 1.35e12
m12f 1.39802e+12 | 1.71e12
m12i 9.7206554e+11 |1.18e12 
m12m 1.2596011e+12 |1.58e12
m12r 9.563382e+11 |1.10e12 
m12w 9.088676e+11 | 1.08e12

In [29]:
m12_subhalos.keys()

dict_keys(['accrete.rate', 'accrete.rate.100Myr', 'accrete.rate.tdyn', 'am.phantom', 'am.progenitor.main', 'axis.b/a', 'axis.c/a', 'descendant.snapshot', 'host.distance', 'host.index', 'host.velocity', 'host.velocity.rad', 'host.velocity.tan', 'id', 'id.to.index', 'infall.first.mass', 'infall.first.snapshot', 'infall.first.vel.circ.max', 'infall.mass', 'infall.snapshot', 'infall.vel.circ.max', 'major.merger.snapshot', 'mass', 'mass.180m', 'mass.200c', 'mass.200m', 'mass.500c', 'mass.bound', 'mass.half.snapshot', 'mass.lowres', 'mass.peak', 'mass.peak.snapshot', 'mass.vir', 'position', 'position.offset', 'progenitor.number', 'radius', 'scale.radius', 'scale.radius.klypin', 'spin.bullock', 'spin.peebles', 'tree.index', 'vel.circ.max', 'vel.circ.peak', 'vel.std', 'velocity', 'velocity.offset', 'dark2.mass', 'star.form.time.100', 'star.form.time.50', 'star.form.time.90', 'star.form.time.95', 'star.form.time.dif.68', 'star.indices', 'star.mass', 'star.massfraction', 'star.number', 'star.pos

In [30]:
np.sort(m12_subhalos['mass.200m'])[::-1]/1e12

array([1.4210399e+00, 7.6367521e-01, 2.0566952e-01, ..., 7.0361821e-08,
       7.0361821e-08, 7.0361821e-08], dtype=float32)

In [2]:
import numpy as np

In [3]:
sim = 'm12b'
snap_times = "/mnt/ceph/users/firesims/fire2/metaldiff/{}_res7100/snapshot_times.txt".format(sim)
times = np.loadtxt(snap_times, usecols=3)

In [16]:
times[492]

11.50138153